In [1]:
import sys

In [2]:
import numpy as np
import pandas as pd
import anndata

In [3]:
from rnasieve.preprocessing import model_from_raw_counts

In [1]:
import rpy2

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [10]:
readRDS = robjects.r['readRDS']

In [11]:
df = readRDS('rnasieve/Xin_bulk_count.rds')

In [12]:
bulk_count = np.array(df)

In [13]:
bulk_count.shape

(6792, 18)

In [14]:
bulk_count[0:4,0:4]

array([[  297.,   269.,   127.,   160.],
       [15067.,  1442.,  5327.,  2453.],
       [ 1488.,   424.,   664.,   994.],
       [ 4199.,  3214.,  4998.,   692.]])

In [15]:
## now create reference dictionary for rnasieve
alpha = readRDS('rnasieve/alpha.rds')
alpha = np.array(alpha)

beta = readRDS('rnasieve/beta.rds')
beta = np.array(beta)

delta = readRDS('rnasieve/delta.rds')
delta = np.array(delta)

gamma = readRDS('rnasieve/gamma.rds')
gamma = np.array(gamma)

In [16]:
## remove genes with all 0's
r1=np.where(np.sum(bulk_count,axis=1)==0)
r2 = np.where(np.sum(alpha,axis=1)==0)
r3 = np.where(np.sum(beta,axis=1)==0)
r4 = np.where(np.sum(delta,axis=1)==0)
r5 = np.where(np.sum(gamma,axis=1)==0)



In [17]:
gamma.shape

(6792, 84)

In [18]:
rm_idx = np.concatenate((r1,r2,r3,r4,r5),axis=1)
rm_idx = np.unique(rm_idx)

In [19]:
bulk_count = np.delete(bulk_count,rm_idx,axis = 0)

In [20]:
alpha = np.delete(alpha,rm_idx,axis = 0)

In [21]:
beta = np.delete(beta,rm_idx,axis = 0)
delta = np.delete(delta,rm_idx,axis = 0)
gamma = np.delete(gamma,rm_idx,axis = 0)

In [22]:
print(bulk_count.shape)
print(alpha.shape)
print(beta.shape)
print(delta.shape)
print(gamma.shape)

(6740, 18)
(6740, 879)
(6740, 470)
(6740, 48)
(6740, 84)


In [23]:
## create dictionary
ref_count = {'alpha':alpha,'beta':beta,'delta':delta,'gamma':gamma}

In [24]:
model1, cleaned_psis1 = model_from_raw_counts(ref_count, bulk_count,trim_percent=0.05,gene_thresh=0.2)

In [25]:
out1 = model1.predict(cleaned_psis1)

/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more inform

In [26]:
out1

,alpha,beta,delta,gamma
Bulk 0,0.560623,0.284382,0.083351,0.071644
Bulk 1,0.145846,0.618863,0.112025,0.123265
Bulk 2,0.603306,0.275760,0.047214,0.073719
Bulk 3,0.421512,0.398234,0.042777,0.137477
Bulk 4,0.465599,0.361569,0.150718,0.022113
Bulk 5,0.678881,0.133365,0.137843,0.049912
Bulk 6,0.206071,0.633506,0.032207,0.128216
Bulk 7,0.295435,0.463923,0.086195,0.154448
Bulk 8,0.603716,0.263908,0.000000,0.132376
Bulk 9,0.411857,0.510260,0.077884,0.000000


In [27]:
import os
os.getcwd()

'/Users/dongyue/Documents/deconvolution'

In [74]:
out1.to_csv('/Users/dongyue/Documents/deconvolution/rnasieve/rnasieve_xin_est.csv')

In [57]:
print((readRDS('rnasieve/Xin_bulk_count_true_beta.rds')))

               alpha       beta       delta       gamma
Non T2D 1  0.7162162 0.17567568 0.067567568 0.040540541
Non T2D 2  0.1739130 0.56521739 0.086956522 0.173913043
Non T2D 3  0.6428571 0.23809524 0.071428571 0.047619048
Non T2D 4  0.5185185 0.37037037 0.000000000 0.111111111
Non T2D 5  0.4423077 0.42307692 0.096153846 0.038461538
Non T2D 6  0.7391304 0.15217391 0.086956522 0.021739130
Non T2D 7  0.2962963 0.55555556 0.000000000 0.148148148
Non T2D 8  0.3333333 0.38095238 0.071428571 0.214285714
Non T2D 9  0.7027027 0.18918919 0.027027027 0.081081081
Non T2D 10 0.4285714 0.52380952 0.047619048 0.000000000
Non T2D 11 0.7931034 0.17241379 0.017241379 0.017241379
Non T2D 12 0.6268657 0.35820896 0.014925373 0.000000000
T2D 1      0.4318182 0.13636364 0.034090909 0.397727273
T2D 2      0.6024096 0.21686747 0.060240964 0.120481928
T2D 3      0.4745763 0.44067797 0.033898305 0.050847458
T2D 4      0.4910394 0.48387097 0.017921147 0.007168459
T2D 5      0.9202454 0.06748466 0.006134969 0.00

In [62]:
out1_ci = model1.compute_marginal_confidence_intervals(sig=0.05)

/Users/dongyue/Documents/deconvolution/rna-sieve/rnasieve/stats.py:129: RuntimeWarning: invalid value encountered in sqrt
  c * np.sqrt(inverse_obs_FI[i, i])) for i in range(alpha.shape[1])]


In [77]:
true_beta = np.array(readRDS('rnasieve/Xin_bulk_count_true_beta.rds'))

In [96]:
print(range(18))

range(0, 18)


In [107]:
coverage = np.zeros((18,4))
for i in range(18):
    b = true_beta[:,i]
    ci = out1_ci[i]
    for j in range(4):
        if b[j]<=ci[j][1] and b[j]>=ci[j][0]:
            coverage[i,j] = 1
    

In [108]:
coverage

array([[0., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 1., 1., 1.],
       [1., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 1., 1., 1.],
       [0., 0., 1., 1.]])

In [112]:
np.mean(coverage,axis=0)

array([0.61111111, 0.83333333, 1.        , 1.        ])

In [114]:
np.round(out1_ci,2)

array([[[ 0.44,  0.68],
        [ 0.17,  0.4 ],
        [ 0.02,  0.15],
        [-0.  ,  0.15]],

       [[ 0.02,  0.27],
        [ 0.47,  0.76],
        [ 0.03,  0.2 ],
        [ 0.04,  0.21]],

       [[ 0.45,  0.75],
        [ 0.13,  0.43],
        [-0.03,  0.13],
        [-0.  ,  0.15]],

       [[ 0.26,  0.58],
        [ 0.24,  0.56],
        [-0.02,  0.1 ],
        [ 0.04,  0.24]],

       [[ 0.18,  0.75],
        [ 0.08,  0.65],
        [-0.02,  0.32],
        [-0.12,  0.16]],

       [[ 0.57,  0.79],
        [ 0.02,  0.25],
        [ 0.05,  0.22],
        [-0.02,  0.12]],

       [[ 0.08,  0.34],
        [ 0.5 ,  0.76],
        [-0.03,  0.09],
        [ 0.04,  0.21]],

       [[ 0.15,  0.44],
        [ 0.32,  0.61],
        [ 0.02,  0.16],
        [ 0.06,  0.25]],

       [[ 0.45,  0.75],
        [ 0.12,  0.4 ],
        [-0.04,  0.04],
        [ 0.05,  0.22]],

       [[ 0.26,  0.57],
        [ 0.37,  0.65],
        [ 0.01,  0.14],
        [-0.06,  0.06]],

       [[ 0.59,  0.8

In [65]:
model1.plot_proportions('stacked').properties(title=".")

alt.Chart(...)

## use selected gene

In [115]:
df = readRDS('rnasieve/Xin_bulk_count_selected.rds')
bulk_count = np.array(df)

In [116]:
alpha = readRDS('rnasieve/alpha_selected.rds')
alpha = np.array(alpha)

beta = readRDS('rnasieve/beta_selected.rds')
beta = np.array(beta)

delta = readRDS('rnasieve/delta_selected.rds')
delta = np.array(delta)

gamma = readRDS('rnasieve/gamma_selected.rds')
gamma = np.array(gamma)

In [117]:
ref_count = {'alpha':alpha,'beta':beta,'delta':delta,'gamma':gamma}

In [118]:
model11, cleaned_psis11 = model_from_raw_counts(ref_count, bulk_count,trim_percent=0.05,gene_thresh=0.2)

In [119]:
out11 = model11.predict(cleaned_psis11)

/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more inform

In [120]:
out11_ci = model11.compute_marginal_confidence_intervals(sig=0.05)

In [121]:
coverage = np.zeros((18,4))
for i in range(18):
    b = true_beta[:,i]
    ci = out1_ci[i]
    for j in range(4):
        if b[j]<=ci[j][1] and b[j]>=ci[j][0]:
            coverage[i,j] = 1

In [122]:
coverage

array([[0., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 1., 1., 1.],
       [1., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 1., 1., 1.],
       [0., 0., 1., 1.]])

In [123]:
np.mean(coverage,axis=0)

array([0.61111111, 0.83333333, 1.        , 1.        ])

In [124]:
np.round(out1_ci,2)

array([[[ 0.44,  0.68],
        [ 0.17,  0.4 ],
        [ 0.02,  0.15],
        [-0.  ,  0.15]],

       [[ 0.02,  0.27],
        [ 0.47,  0.76],
        [ 0.03,  0.2 ],
        [ 0.04,  0.21]],

       [[ 0.45,  0.75],
        [ 0.13,  0.43],
        [-0.03,  0.13],
        [-0.  ,  0.15]],

       [[ 0.26,  0.58],
        [ 0.24,  0.56],
        [-0.02,  0.1 ],
        [ 0.04,  0.24]],

       [[ 0.18,  0.75],
        [ 0.08,  0.65],
        [-0.02,  0.32],
        [-0.12,  0.16]],

       [[ 0.57,  0.79],
        [ 0.02,  0.25],
        [ 0.05,  0.22],
        [-0.02,  0.12]],

       [[ 0.08,  0.34],
        [ 0.5 ,  0.76],
        [-0.03,  0.09],
        [ 0.04,  0.21]],

       [[ 0.15,  0.44],
        [ 0.32,  0.61],
        [ 0.02,  0.16],
        [ 0.06,  0.25]],

       [[ 0.45,  0.75],
        [ 0.12,  0.4 ],
        [-0.04,  0.04],
        [ 0.05,  0.22]],

       [[ 0.26,  0.57],
        [ 0.37,  0.65],
        [ 0.01,  0.14],
        [-0.06,  0.06]],

       [[ 0.59,  0.8

In [134]:
np.sqrt(np.mean(np.square(np.array(out11 - np.transpose(true_beta)))))

0.07726258449893357

In [66]:
## arbitrary proportions
bulk_count_arb = readRDS('rnasieve/Xin_bulk_count_arbitrary_prop.rds')
bulk_count_arb = np.array(bulk_count_arb)
bulk_count_arb = np.delete(bulk_count_arb,rm_idx,axis = 0)

In [67]:
model2, cleaned_psis2 = model_from_raw_counts(ref_count, bulk_count_arb,trim_percent=0.05,gene_thresh=0.2)

In [68]:
out2 = model2.predict(cleaned_psis2)

/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more inform

In [69]:
out2

,alpha,beta,delta,gamma
Bulk 0,0.316558,0.377991,0.190861,0.114589
Bulk 1,0.378962,0.348972,0.199963,0.072103
Bulk 2,0.370825,0.347346,0.163180,0.118649
Bulk 3,0.424094,0.265890,0.244512,0.065504
Bulk 4,0.374667,0.321262,0.243584,0.060487
Bulk 5,0.359752,0.376481,0.167832,0.095936
Bulk 6,0.416094,0.456097,0.000679,0.127131
Bulk 7,0.490968,0.422806,0.051844,0.034382
Bulk 8,0.037625,0.099506,0.247281,0.615588
Bulk 9,0.131090,0.106160,0.279558,0.483192


In [36]:
print(readRDS('rnasieve/Xin_bulk_count_arbitrary_prop_true_beta.rds'))

           alpha beta delta gamma
Non T2D 1    0.4  0.3   0.2   0.1
Non T2D 2    0.4  0.3   0.2   0.1
Non T2D 3    0.4  0.3   0.2   0.1
Non T2D 5    0.4  0.3   0.2   0.1
Non T2D 6    0.4  0.3   0.2   0.1
Non T2D 8    0.4  0.3   0.2   0.1
Non T2D 9    0.4  0.3   0.2   0.1
Non T2D 11   0.4  0.2   0.2   0.2
T2D 1        0.4  0.2   0.2   0.2
T2D 2        0.4  0.2   0.2   0.2
T2D 3        0.4  0.2   0.2   0.2
T2D 4        0.4  0.2   0.2   0.2
T2D 5        0.4  0.2   0.2   0.2
T2D 6        0.4  0.2   0.2   0.2



In [70]:
out2_ci = model2.compute_marginal_confidence_intervals(sig=0.05)

/Users/dongyue/Documents/deconvolution/rna-sieve/rnasieve/stats.py:129: RuntimeWarning: invalid value encountered in sqrt
  c * np.sqrt(inverse_obs_FI[i, i])) for i in range(alpha.shape[1])]


In [71]:
out2_ci

[[(nan, nan),
  (nan, nan),
  (0.09623050954255237, 0.2854915579784694),
  (-0.019045230932589086, 0.24822414068580556)],
 [(nan, nan),
  (nan, nan),
  (0.094424840160179, 0.30550202632426104),
  (-0.017276645168872395, 0.1614816600848548)],
 [(nan, nan),
  (nan, nan),
  (0.046651043386650326, 0.27970877250697507),
  (0.007600110707986829, 0.22969780789051716)],
 [(nan, nan),
  (nan, nan),
  (0.1049888898849877, 0.38403558475638055),
  (-0.04617031147704459, 0.17717761865188464)],
 [(nan, nan),
  (nan, nan),
  (0.11301917650604554, 0.37414930529341983),
  (-0.07130911085255687, 0.1922821877745889)],
 [(nan, nan),
  (nan, nan),
  (0.04295655830670207, 0.29270817806201305),
  (-0.0343530996970729, 0.2262242418261246)],
 [(nan, nan),
  (nan, nan),
  (-0.026412922828547937, 0.027769927047901974),
  (0.01922748918693032, 0.23503485984079767)],
 [(nan, nan),
  (nan, nan),
  (-0.016411513945121955, 0.1200996299358567),
  (-0.06665957436886885, 0.13542377038987394)],
 [(-0.020913541384780424, 

In [29]:
import pandas as pd
object = pd.read_pickle("rnasieve/pydict")

In [30]:
object

{'Python': array([[0.05435182, 0.0281588 , 0.04076225, 0.04059642, 0.02307307],
        [1.38620197, 2.10199706, 0.88167447, 1.32144163, 1.10771262],
        [0.06067014, 0.05411912, 0.02008042, 0.06230577, 0.04260076],
        [0.10053089, 0.28033634, 0.0632551 , 0.11781591, 0.11448539],
        [3.08324976, 0.50175644, 2.31978051, 0.66613204, 1.24934222]]),
 'C++': '.cpp',
 'Java': '.java'}

In [54]:
temp = pd.read_csv('rnasieve/temp.txt',sep="\t",index_col=0)

In [55]:
temp

,delta,alpha,delta.1,gamma,alpha.1
gene,,,,,
SGIP1,0,0,0,32,0
AZIN2,0,0,0,0,0
CLIC4,3,0,0,1,0
AGBL4,0,0,0,0,0
NECAP2,0,0,0,0,0


In [56]:
temp.values

array([[ 0,  0,  0, 32,  0],
       [ 0,  0,  0,  0,  0],
       [ 3,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0]])

In [48]:
np.genfromtxt('rnasieve/temp.txt',skip_header=1)

array([[nan,  0.,  0.,  0., 32.,  0.],
       [nan,  0.,  0.,  0.,  0.,  0.],
       [nan,  3.,  0.,  0.,  1.,  0.],
       [nan,  0.,  0.,  0.,  0.,  0.],
       [nan,  0.,  0.,  0.,  0.,  0.]])